# Assignment 3 Group no. 10
### Project members: 
Veronika Cucorova <cucorova@kth.se> 

Tim Roelofs <tjtro@kth.se> 

Léo Vuylsteker <leov@kth.se> 

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All assignments starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of assignments starting with 
number 2 below, then the assignment will receive 2 points (in total).

It is highly recommended that you do not develop the code directly within the notebook
but that you copy the comments and test cases to your regular development environment
and only when everything works as expected, that you paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).


## Load NumPy, pandas and time

In [1]:
import numpy as np
import pandas as pd
import time


## Reused functions from Assignment 1

In [2]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

def create_normalization(df, normalizationtype="minmax"):
    #   we need to do the modifications on a deep copy of the dataframe 
    deep_copy_df = df.copy(deep=True)
    if normalizationtype == 'minmax':
        #   define the min and max function on a dataframe
        function1 = lambda dataframe: dataframe.min()
        function2 = lambda dataframe: dataframe.max()
    elif normalizationtype == 'zscore':
        #   define the mean and std function on a dataframe
        function1 = lambda dataframe: dataframe.mean()
        function2 = lambda dataframe: dataframe.std()
    else:
        #   if the keyword is not recognize, we make sure that the programme raises an error to indicate
        #   where the problem is
        raise ValueError('The normalizationtype variable has not been recognized. Please use either minmax '
                         'or zscore.')
    #   we remove the CLASS and ID columns, which have special meanings, from the working dataframe
    wrk_df = deep_copy_df[deep_copy_df.columns.difference(['CLASS', 'ID'])]
    #   we select the columns with contain numeric types
    wrk_df = wrk_df.select_dtypes(np.number)
    dict = {}
    for key in wrk_df.columns:
        #   we add the column name to the dictionary keys along with the corresponding normalization
        dict[key] = (normalizationtype, function1(wrk_df[key]), function2(wrk_df[key]))
        if normalizationtype == 'minmax':
            #   we apply the corresponding min-max normalization to the column
            deep_copy_df[key] = deep_copy_df[key].map(
                lambda x: (x-function1(wrk_df[key]))/(function2(wrk_df[key] - function1(wrk_df[key])))
            )
        else:
            #   we apply the corresponding z-normalization to the column
            deep_copy_df[key] = deep_copy_df[key].map(
                lambda x: (x-function1(wrk_df[key]))/function2(wrk_df[key])
            )
    return deep_copy_df, dict


def apply_normalization(df, normalization):
    #   we need to do the modifications on a deep copy of the dataframe 
    deep_copy_df = df.copy(deep=True)
    for key in normalization.keys():
        normalizationtype = normalization[key][0]
        if normalizationtype == 'minmax':
            min = normalization[key][1]
            max = normalization[key][2]
            #   we apply the corresponding min-max normalization to the column
            deep_copy_df[key] = deep_copy_df[key].map(
                lambda x: (x-min)/(max-min)
            )
        elif normalizationtype == 'zscore':
            #   we apply the corresponding z-normalization to the column
            mean = normalization[key][1]
            std = normalization[key][2]
            deep_copy_df[key] = deep_copy_df[key].map(
                lambda x: (x-mean)/std
            )
        else:
            #   if the keyword is not recognize, we make sure that the programme raises an error to indicate
            #   where the problem is
            raise ValueError('The normalizationtype variable has not been recognized. Please use either'
                             ' minmax or zscore.')
    return deep_copy_df


def create_imputation(df):
    wrk_df = df.copy()
    dict = {}
    columns = wrk_df.columns.difference(['CLASS', 'ID'])
    for key in columns:
        #treat column as numeric
        if (wrk_df[key].dtypes == np.float64 or wrk_df[key].dtypes == np.int64):
            #first thing that came to my mind on checking if all values are null
            if (wrk_df[key].isnull().sum() == len(wrk_df[key])):
                repl_val = 0
            else:
                repl_val = wrk_df[key].mean()          
        #treat value as categorical
        elif (wrk_df[key].dtypes == np.bool or wrk_df[key].dtypes.name == 'category'):
            if (wrk_df[key].isnull().sum() == len(wrk_df[key])):
                repl_val = wrk_df[key].categories[0]
            else: 
                repl_val = wrk_df[key].mode()[0]
        #treat value as a string
        elif (wrk_df[key].dtypes == np.object):
            if (wrk_df[key].isnull().sum() == len(wrk_df[key])):
                repl_val = ""
            else: 
                repl_val = wrk_df[key].mode()[0] #mode can return multiple things
        else: 
            raise ValueError('Unknown column type.')

        dict[key] = (repl_val)
        wrk_df[key].fillna(repl_val, inplace = True)
    return wrk_df, dict


def apply_imputation(df, repl_vals):
    wrk_df = df.copy()
    for key in repl_vals:
        wrk_df[key].fillna(repl_vals[key], inplace = True)
    return wrk_df


def create_bins(df, nobins=10, bintype="equal-width"):
    newdf = df.copy()
    binning = {}
    binfunctions = {'equal-width':pd.cut, 'equal-size':pd.qcut}
    for key in newdf.columns:
        if newdf[key].dtype == np.number and key not in ['CLASS', 'ID']:
            res, bins = binfunctions[bintype](newdf[key], nobins, labels=False, retbins=True, duplicates="drop")
            newdf[key] = res
            #hint 6 implementation. This part assumes that there are more than 2 bins
            bins[0] = -np.inf
            bins[-1] = np.inf 
            
            binning[key] = bins
            
    #Hint 4 implementation
    for key in newdf.columns:
        newdf[key] = newdf[key].astype('category')
    
    return newdf, binning


def apply_bins(df, binning):
    newdf = df.copy()
    for key in newdf.columns:
        if key in binning:
            newdf[key] = pd.cut(df[key],binning[key], labels=False)
    
    #Hint 3 implementation
    for key in newdf.columns:
        newdf[key] = newdf[key].astype('category')
    
    return newdf


def split(df, testfraction=0.5):
    #   generate a permutation of the row numbers
    permutation = np.random.permutation(df.index)
    #   compute the index separating the testing rows from the training rows in permutation
    sep_index = int(permutation.size*testfraction)
    #   return two slices of the initial data set: 
    #   the first one contains 1-testfraction fraction of the dataframe and correspond to the training set
    #   and the other one contains testfraction fraction of the dataframe and correspond to the testing set.
    return df.iloc[permutation[sep_index:], :], df.iloc[permutation[:sep_index], :]


def accuracy(preds, labels):
    max_prob = []
    max_prob = preds.idxmax(axis=1)
    #perform pairwise comparison of predicted with actual labels
    compared = np.equal(max_prob, labels)
    #return the number of true ones
    return(len(compared[compared == True])/len(compared))


def create_one_hot(df):
    newdf = pd.DataFrame()
    one_hot = {}
    for key in df.columns:
        if df[key].dtype in [np.object] and key not in ['CLASS', 'ID']:
            one_hot[key] = df[key].unique()
            newtab = pd.get_dummies(df[key], prefix=key)
            for i in newtab.columns: #converting to floats
                newtab[i] = newtab[i].astype(float)
            newdf = pd.concat([newdf, newtab], axis=1)
        else:
            newdf = pd.concat([newdf, df[key]], axis=1)
    
    return(newdf, one_hot)


def apply_one_hot(df, one_hot):
    newdf = pd.DataFrame()
    for key in df.columns:
        if key in one_hot:
            cols = pd.CategoricalDtype(categories=one_hot[key])
            newcol = df[key].astype(cols)
            newtab = pd.get_dummies(newcol, prefix=key)
            for i in newtab.columns: #converting to floats
                newtab[i] = newtab[i].astype(float)
                
            newdf = pd.concat([newdf, newtab], axis=1)
        else:
            newdf = pd.concat([newdf, df[key]], axis=1)
    return(newdf)


def folds(df, nofolds=10):
    #   generate a permutation of the row numbers
    permutation = np.random.permutation(df.index)
    #   split the permutation into equal-sized subsets of row numbers
    folds_indexes = np.array_split(permutation, nofolds)
    #   return slices of the data set according to the previous subsets of row numbers
    return [df.iloc[indexes, :] for indexes in folds_indexes]


def brier_score(preds, labels):
    #create boolean mask for each label and assign it as an array to the dict
    data = {l:np.array(labels) == l for l in list(preds.columns)}
    labels_df = pd.DataFrame(data)    
    wrk_df = (labels_df - preds)**2
    sum_ = wrk_df.sum().sum()
    n = wrk_df.shape[0]
    return(sum_/n)


def auc(df, correctlabels):
    correctlabels = list(correctlabels)
    finauc = 0
    freq = {}
    aucdict = {}
    for c in set(correctlabels):
        freq[c] = correctlabels.count(c)
        y_true = np.array([1 if x is c else 0 for x in correctlabels])
        scores = df[c]
        scoredct = {}
        poscount = scores[[bool(x) for x in y_true]].value_counts()
        negcount = scores[[not bool(x) for x in y_true]].value_counts()
        for score in sorted(set(scores), reverse=True):
            posscore = poscount[score] if score in poscount else 0
            negscore = negcount[score] if score in negcount else 0
            scoredct[score] = (posscore, negscore)

        tot_tp = sum([scoredct[key][0] for key in scoredct])
        tot_fp = sum([scoredct[key][1] for key in scoredct])
        auc = 0
        cov_tp = 0
        for i in scoredct:
            tp_i = scoredct[i][0]
            fp_i = scoredct[i][1]
            if fp_i == 0:
                cov_tp += tp_i
            elif tp_i == 0:
                auc += (cov_tp/tot_tp)*(fp_i/tot_fp)
            else:
                auc += (cov_tp/tot_tp)*(fp_i/tot_fp) + (tp_i/tot_tp)*(fp_i/tot_fp)/2
                cov_tp += tp_i
        
        aucdict[c] = auc       
    for key in aucdict:
        finauc += freq[key]/len(correctlabels)*aucdict[key]
    return finauc



## 1. Define the class DecisionTree

In [3]:
# Define the class DecisionTree with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
# self.model should be a decision tree (for details, see lecture slides), where the leafs return class probabilities
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First find the available features (excluding "CLASS" and "ID"), then find the class counts, e.g., using 
#         groupby, and calculate the default class probabilities (relative frequencies of the class labels)
#
# Hint 2: Define a function, e.g., called divide_and_conquer, that takes the above as input together with df 
#         and min_samples_split, and also a nodeno (starting with 0) to keep track of the generated nodes in the tree
#
# Hint 3: You may represent the tree under construction as a list of nodes (tuples), on the form:
#         (nodeno,"leaf",class_probabilities): corresponding to a leaf node where class_probabilities is a vector
#                                              with the relative class frequencies (ordered according to self.labels)
#         (nodeno,feature,node_dict): corresponding to an internal (non-leaf) node where node_dict is a mapping from
#                                     the possible values of feature to child nodes (their nodenos)
#
# Hint 4: You may evaluate each feature by a function information_content, which takes the group sizes
#         for each possible value of the feature together with the class counts of each group as input
#
# Hint 5: The best feature found (with lowest residual information content) will be used to split the training
#         instances, and each sub-group will be used for generating a sub-tree (recursively by divide_and_conquer,
#         see lecture slides for details)
#
# Hint 6: You may make divide_and_conquer return not only a list of nodes, but also a current_node_no; 
#         by this, each subsequent call to divide_and_conquer for each subset of instances, i.e. for each feature value, 
#         could use current_node_no as a starting point.
#         If you e.g. make the following call:
#
#         current_node_no, node_list = divide_and_conquer(current_node_no, ...)
#
#         then the returned value in current_node_no can be used in the next call to divide_and_conquer.
#         Node that node_list will contain an arbitrary number of tuples, each element corresponding to a node together 
#         with a node number. The first element in the list will have the same number as current_node_no when the call 
#         was made and the last element will have a number one less than current_node_no when returned, e.g., if there is
#         only one (leaf) node in the returned list, then current_node_no will only be incremented by one through the above call.
#         
# Hint 7: The list of nodes output by divide_and_conquer may finally be converted to an array, where each nodeno in the 
#         tuples corresponds to an index of the array
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the relative class frequencies in the leaves of the decision tree into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(nodeno,row), which for a test row
#         finds a leaf node from which class probabilities are obtained
# Hint 3: This sub-function may recursively traverse the tree (represented by an array), starting with the nodeno
#         that corresponds to the root


class DecisionTree:
    
    def ___init___(self):
        self.binning = None
        self.imputation = None
        self.labels = None
        self.model = None
    
    def fit(self, df, nobins=10, bintype='equal-width', min_samples_split=5):
        #   We modify the dataset df in order to fill missing values and only have categorical features
        df, self.imputation = create_imputation(df)
        df, self.binning = create_bins(df, nobins, bintype) 
        self.labels = df["CLASS"].astype('category')
        
        #   We compute the class frequencies of this modified df
        freqdict = {}
        grouped = df.groupby('CLASS')
        for key in self.labels.unique():
            try:
                freqdict[key] = len(grouped.get_group(key))/len(df)
            except:
                freqdict[key] = 0.0
        self.model = {}
        
        #   The index of the root is set to 0; we will store it as a global variable to keep track of the most recently
        #   created node and avoid index issues in divide_and_conquer
        self.nodeno = 0
        
        #   We call the recursive function divide_and_conquer to create the nodes of our model step by step
        self.divide_and_conquer(freqdict, df, min_samples_split, self.nodeno)



    def divide_and_conquer(self, relative_frequencies, df, min_samples_split, nodeno):
        #   stop criterion n°1 : the current data frame contains less than min_samples_slit training samples 
        if len(df) <= min_samples_split:
            #   the current node is then considered as a leaf
            self.model[nodeno] = ('leaf', relative_frequencies)
            return
        
        #   next step: find the subsets which minimize the residual info 
        #   and the corresponding feature used for splitting
        min_info_res = float('inf')
        best_subsets = None
        feature = None
        for c in df.columns:
            if c not in ["ID", "CLASS"]:
                #   we do that by trying each feature to do the split
                ir, subsets = self.information_content(c, df)
                if ir < min_info_res:
                    min_info_res = ir
                    best_subsets = subsets
                    feature = c
        
        #   stop criterion n°2 : the current data frame does not have any feature left to split data
        if feature is None:
            #   the current node is then considered as a leaf
            self.model[nodeno] = ('leaf', relative_frequencies)
            return
        
        #   now we need to create the corresponding node which is not a leaf
        #   this means that we need to split the data frame according to the selected feature
        #   and then call recursively the function on each subset 
        #   (and obviously update the values of df, relative_frequency and nodeno)
        
        #   In the mean time we will create the dictionary storing the children of the current node
        node_dict = {}
        values = df[feature].unique()  # unique values of the feature
        allpossvalues = len(self.binning[feature])-1
        
        for value in range(allpossvalues):
            self.nodeno += 1  # we are adding a new child node => we need to update our global nodeno to avoid conflicts
            node_dict[value] = self.nodeno  # the child node will be created at the next available nodeno
            if value in values:
                subset = best_subsets.get_group(value)  # the subset corresponding to the child node
                del subset[feature]  # we already used this feature, by deleting the corresponding column we avoid useless checking later on
                #   We compute the class frequencies of this modified df
                grouped = subset.groupby('CLASS')
                freqdict = {}
                for key in self.labels.unique():
                    try:
                        freqdict[key] = len(grouped.get_group(key))/len(subset)
                    except:
                        freqdict[key] = 0.0
                #   we call the function again on this new child node (Depth-First Search)
                self.divide_and_conquer(freqdict, subset, min_samples_split, self.nodeno)
            else:
                self.model[self.nodeno] = ('leaf', relative_frequencies)
        
        #   finally, we create the current node with the node_dict which has just been created
        self.model[nodeno] = (feature, node_dict)
    
    
    
    
    def information_content(self, feature, I_tot):
        feature_values = I_tot[feature].unique()   # unique values of the feature
        grouped_I_tot = I_tot.groupby(feature)   # df grouped by feature values
        res_info = 0    # residual info of the split according to the feature
        for value in feature_values:
            I_value = grouped_I_tot.get_group(value)    # subset of I_tot where feature = value
            weight = len(I_value)/len(I_tot)  # |I_value| / |I_tot|
            classes = I_value['CLASS'].unique()    # unique classes of I_value subset
            grouped_I_value = I_value.groupby('CLASS')   # I_value subset grouped by value
            info_sum = 0
            for _class in classes:
                p = len(grouped_I_value.get_group(_class))/len(grouped_I_value)  # P(Class = _class) in I_value subset
                info_sum -= p * np.log2(p)
            res_info += weight * info_sum
            
        return res_info, grouped_I_tot
    
    def predict(self, df):
        df = df.drop(columns=['CLASS', 'ID'])
        df = apply_imputation(df, self.imputation)
        df = apply_bins(df, self.binning)
        predictions = pd.DataFrame(columns=self.labels.unique())
        for index, row in df.iterrows():
            outcome = self.make_prediction(0, row)
            predictions = pd.concat([predictions, outcome], ignore_index=True)
        return(predictions)
    
    def make_prediction(self, nodeno,row):
        if self.model[nodeno][0] == 'leaf':
            prediction = pd.DataFrame.from_records([self.model[nodeno][1]])
            return prediction
        else:
            newnode = self.model[nodeno][1][row[self.model[nodeno][0]]]
            prediction = self.make_prediction(newnode, row)
            return prediction

In [4]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

tree_model = DecisionTree()

test_labels = glass_test_df["CLASS"]

nobins_values = [5,10]
bintype_values = ["equal-width","equal-size"]
min_samples_split_values = [3,5,10]
parameters = [(nobins,bintype,min_samples_split) for nobins in nobins_values for bintype in bintype_values 
              for min_samples_split in min_samples_split_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    tree_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1],min_samples_split=parameters[i][2])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = tree_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values,min_samples_split_values]),
                       columns=["Accuracy","Brier score","AUC"])

results



Training time (5, 'equal-width', 3): 2.47 s.
Testing time (5, 'equal-width', 3): 0.20 s.
Training time (5, 'equal-width', 5): 2.04 s.
Testing time (5, 'equal-width', 5): 0.15 s.
Training time (5, 'equal-width', 10): 1.51 s.
Testing time (5, 'equal-width', 10): 0.15 s.
Training time (5, 'equal-size', 3): 3.03 s.
Testing time (5, 'equal-size', 3): 0.15 s.
Training time (5, 'equal-size', 5): 2.45 s.
Testing time (5, 'equal-size', 5): 0.14 s.
Training time (5, 'equal-size', 10): 1.93 s.
Testing time (5, 'equal-size', 10): 0.14 s.
Training time (10, 'equal-width', 3): 3.76 s.
Testing time (10, 'equal-width', 3): 0.15 s.
Training time (10, 'equal-width', 5): 2.59 s.
Testing time (10, 'equal-width', 5): 0.19 s.
Training time (10, 'equal-width', 10): 2.37 s.
Testing time (10, 'equal-width', 10): 0.14 s.
Training time (10, 'equal-size', 3): 3.81 s.
Testing time (10, 'equal-size', 3): 0.14 s.
Training time (10, 'equal-size', 5): 3.71 s.
Testing time (10, 'equal-size', 5): 0.17 s.
Training time (

Accuracy  Brier score       AUC
5  equal-width 3   0.532710     0.606309  0.751842
               5   0.616822     0.585608  0.766284
               10  0.588785     0.623317  0.739990
   equal-size  3   0.607477     0.645337  0.766617
               5   0.607477     0.641879  0.758390
               10  0.598131     0.562977  0.789628
10 equal-width 3   0.560748     0.646725  0.770803
               5   0.560748     0.641512  0.767954
               10  0.570093     0.590358  0.777335
   equal-size  3   0.420561     1.029004  0.610476
               5   0.420561     1.024331  0.610628
               10  0.457944     0.779814  0.637095

In [5]:
train_labels = glass_train_df["CLASS"]
tree_model.fit(glass_train_df,min_samples_split=1)
predictions = tree_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.97
AUC on training set: 1.00
Brier score on training set: 0.03


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class DecisionForest

In [6]:
# Define the class DecisionForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
# random_features: no. of features to evaluate at each split (default = 2), 0 means all features (no random sampling)
# notrees: no. of trees in the forest (default = 10)
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
# self.model should be a random forest (for details, see lecture slides)
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: Redefine divide_and_conquer to take one additional argument; random_features, and instead of
#         evaluating all features choose a random subset, e.g., by np.random.choice (without replacement)
# Hint 2: Generate each tree in the forest from a bootstrap replicate of df, e.g., by np.random.choice 
#         (with replacement) from the index values of df.
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the mean of all relative class frequencies in the leaves of the forest into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(row), which for a test row
#         finds all leaf nodes and calculates the average of their class probabilities


class DecisionForest:
    
    def ___init___(self):
        self.binning = None
        self.imputation = None
        self.labels = None
        self.model = None
    
    def fit(self, df, nobins=10, bintype='equal-width', min_samples_split=5, random_features=2, notrees=10):
        #   We modify the dataset df in order to fill missing values and only have categorical features
        df, self.imputation = create_imputation(df)
        df, self.binning = create_bins(df, nobins, bintype) 
        self.labels = df["CLASS"].astype('category')
        
        #   We compute the class frequencies of this modified df
        freqdict = {}
        grouped = df.groupby('CLASS')
        for key in self.labels.unique():
            try:
                freqdict[key] = len(grouped.get_group(key))/len(df)
            except:
                freqdict[key] = 0.0
        self.model = []
        
        for index in range(notrees):
            self.tree = {}
            
            #   The index of the root is set to 0; we will store it as a global variable to keep track of the most recently
            #   created node and avoid index issues in divide_and_conquer
            self.nodeno = 0

            #   We call the recursive function divide_and_conquer to create the nodes of our model step by step
            self.divide_and_conquer(freqdict, df, min_samples_split, self.nodeno, random_features)
            self.model.append(self.tree)
    
    
    
    
    
    def divide_and_conquer(self, relative_frequencies, df, min_samples_split, nodeno, random_features):
        #   stop criterion n°1 : the current data frame contains less than min_samples_slit training samples 
        if len(df) <= min_samples_split:
            #   the current node is then considered as a leaf
            self.tree[nodeno] = ('leaf', relative_frequencies)
            return
        
        #   next step: find the subsets which minimize the residual info 
        #   and the corresponding feature used for splitting
        min_info_res = float('inf')
        best_subsets = None
        feature = None
        
        random_columns = [col for col in df.columns.values if col not in ['CLASS', 'ID']]
        np.random.shuffle(random_columns)
        if(random_features > 0 and random_features < len(random_columns)-2):
            random_columns = random_columns[:random_features]
            
        for c in random_columns:
            #   we do that by trying each feature to do the split
            ir, subsets = self.information_content(c, df)
            if ir < min_info_res:
                min_info_res = ir
                best_subsets = subsets
                feature = c
        
        #   stop criterion n°2 : the current data frame does not have any feature left to split data
        if feature is None:
            #   the current node is then considered as a leaf
            self.tree[nodeno] = ('leaf', relative_frequencies)
            return
        
        #   now we need to create the corresponding node which is not a leaf
        #   this means that we need to split the data frame according to the selected feature
        #   and then call recursively the function on each subset 
        #   (and obviously update the values of df, relative_frequency and nodeno)
        
        #   In the mean time we will create the dictionary storing the children of the current node
        node_dict = {}
        values = df[feature].unique()  # unique values of the feature
        allpossvalues = len(self.binning[feature])-1
        
        for value in range(allpossvalues):
            self.nodeno += 1  # we are adding a new child node => we need to update our global nodeno to avoid conflicts
            node_dict[value] = self.nodeno  # the child node will be created at the next available nodeno
            if value in values:
                subset = best_subsets.get_group(value)  # the subset corresponding to the child node
                del subset[feature]  # we already used this feature, by deleting the corresponding column we avoid useless checking later on
                #   We compute the class frequencies of this modified df
                grouped = subset.groupby('CLASS')
                freqdict = {}
                for key in self.labels.unique():
                    try:
                        freqdict[key] = len(grouped.get_group(key))/len(subset)
                    except:
                        freqdict[key] = 0.0
                #   we call the function again on this new child node (Depth-First Search)
                self.divide_and_conquer(freqdict, subset, min_samples_split, self.nodeno, random_features)
            else:
                self.tree[self.nodeno] = ('leaf', relative_frequencies)
        
        #   finally, we create the current node with the node_dict which has just been created
        self.tree[nodeno] = (feature, node_dict)
    
    
    
    
    
    def information_content(self, feature, I_tot):
        feature_values = I_tot[feature].unique()   # unique values of the feature
        grouped_I_tot = I_tot.groupby(feature)   # df grouped by feature values
        res_info = 0    # residual info of the split according to the feature
        for value in feature_values:
            I_value = grouped_I_tot.get_group(value)    # subset of I_tot where feature = value
            weight = len(I_value)/len(I_tot)  # |I_value| / |I_tot|
            classes = I_value['CLASS'].unique()    # unique classes of I_value subset
            grouped_I_value = I_value.groupby('CLASS')   # I_value subset grouped by value
            info_sum = 0
            for _class in classes:
                p = len(grouped_I_value.get_group(_class))/len(grouped_I_value)  # P(Class = _class) in I_value subset
                info_sum -= p * np.log2(p)
            res_info += weight * info_sum
            
        return res_info, grouped_I_tot
    
    def predict(self, df):
        None
    
    def make_prediction(self, row):
        return None


In [7]:
glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

forest_model = DecisionForest()

test_labels = glass_test_df["CLASS"]

min_samples_split_values = [1,2,5]
random_features_values = [1,2,5]

parameters = [(min_samples_split,random_features) for min_samples_split in min_samples_split_values 
              for random_features in random_features_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    forest_model.fit(glass_train_df,min_samples_split=parameters[i][0],random_features=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = forest_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([min_samples_split_values,random_features_values]),
                       columns=["Accuracy","Brier score","AUC"])

results

Training time (1, 1): 16.22 s.
Testing time (1, 1): 0.00 s.


AttributeError: 'NoneType' object has no attribute 'idxmax'

In [24]:
train_labels = glass_train_df["CLASS"]
forest_model.fit(glass_train_df,min_samples_split=1)
predictions = forest_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.96
AUC on training set: 1.00
Brier score on training set: 0.12


### Comment on assumptions, things that do not work properly, etc.